# 박상준 코드

#### 목적 : 각 list의 element기반으로 영화 데이터를 추출하는 코드
    - 감독, 제목 & 출시년도 & 장르세트, 평점, 원제(있는것만), 평점(후순위)

## Develop Log
- Code Created by 박상준
- Code Developed by 강태훈

In [5]:
import pandas as pd
import time
from datetime import datetime
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # webdriver-manager 패키지 다운로드
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from tqdm import tqdm

In [6]:
def selenium_chrome(url='about:blank', headless_mode=False):
    '''
    Selenium 시작하는 함수
    
    :param url: 접속할 링크
    :param headless_mode: Headless 실행여부
    
    :return driver: Selenium Driver Object
    '''
    
    if headless_mode is True:
        # 옵션 추가 
        options = webdriver.ChromeOptions() 
        options.add_argument('window-size=1920x1080')
        options.add_argument('disable-gpu')
        options.add_argument('lang=ko_KR')
        options.add_argument('headless')           # 이 부분이 HEADLESS실행하는 옵션!! 나머지는 옵션들
        options.add_argument("no-sandbox")
        options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.80 Safari/537.36')
        options.add_argument('ignore-certificate-errors')
        options.add_argument('allow-running-insecure-content')
        # 개인 속성창 확인 url : 'https://intoli.com/blog/making-chrome-headless-undetectable/chrome-headless-test.html'

        driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
        driver.set_window_size(1920, 1080)
        # driver.set_window_position(0, -2000) # 해당 코드 실행 시 selenium브라우저가 좌표값 (x: 0, y:-2000) 위치에 시작
        driver.get('about:blank')    # about:blank는 그냥 아무것도 없는 페이지 띄워놓겠다는 것
        driver.execute_script("Object.defineProperty(navigator, 'plugins', {get: function() {return[1, 2, 3, 4, 5];},});")
        driver.execute_script("Object.defineProperty(navigator, 'languages', {get: function() {return ['ko-KR', 'ko', 'en-US', 'en']}})")
        driver.execute_script("const getParameter = WebGLRenderingContext.getParameter;WebGLRenderingContext.prototype.getParameter = function(parameter) {if (parameter === 37445) {return 'Intel Open Source Technology Center'} if (parameter === 37446) {return 'Mesa DRI Intel(R) Ivybridge Mobile';}return getParameter(parameter);};")
    
    elif headless_mode is False:
        driver = webdriver.Chrome(ChromeDriverManager().install())
        # driver.set_window_size(1920, 1080)
        driver.get('about:blank')
        
    driver.get(url)
    return driver

In [7]:
def driver_wait_XPATH(driver, XPATH, url='about:blank', waitingtime=10):
    '''"
    목적1) : 지정한 ID element값이 나타날때까지 대기시키는 함수
    목적2) : 일정시간을 초과한 경우 selenium을 다시 재실행 시키기 -> 신규 쿠키로 활동시작
    
    :param driver: selenium Driver
    :param XPATH: element의 XPATH값
    :param waitingtime: 최대 대기시간
    
    :return: driver
    '''
    
    try: 
        element = WebDriverWait(driver, waitingtime).until(
            EC.presence_of_element_located((By.XPATH, XPATH))
        )                          # BY.XPATH는 타입이 XPATH, 두번째 XPATH는 XPATH 경로를 정의해줄 것임. EC는 위에서 import 한 건데, 원하는 게 나올 때까지 기다린다는 것임.
    
    #예외 처리 기능 : 해당 xpath가 없다면 TimeoutException을 실행함.
    except TimeoutException:
        driver.quit()
        driver = selenium_chrome(url, headless_mode=True)
        time.sleep(2)
        
        # TimeOut에러말고 다른 에러를 e로 받아와서 프린트해줌.
    except Exception as e:
        print(e)
    
    return driver

In [ ]:
# 더보기 페이지로 이동
driver.get(OverviewURL)

# 기본정보 element나올때 까지 대기시키기. headless는 클릭기능x. url 새로 넣어서 OverviewURL이거로 들어감 이건 그 사이트에서 알아서.
driver = driver_wait_XPATH(driver, XPATH = '//*[@id="root"]/div/div[1]/section/section/header/div[2]/div', url=OverviewURL)